In [727]:
# Load the "autoreload" extension so that code can change
%load_ext autoreload

# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [728]:
import tensorflow as tf

In [729]:
tf.keras.preprocessing.text.Tokenizer()

In [730]:
import tensorflow_hub as hub

In [787]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50-with-normalization/1",
                   dtype=tf.string,
                   input_shape=[],
                   output_shape=[50])])

In [788]:
model.predict(x)

ValueError: Error when checking input: expected keras_layer_19_input to have 1 dimensions, but got array with shape (2, 6)

In [1014]:
build_embeddings_from_vocab()

<tf.Tensor: shape=(1, 50), dtype=float32, numpy=
array([[-0.16658899, -0.026856  , -0.23500197,  0.16764398, -0.027478  ,
        -0.05976099,  0.21726097, -0.14739199, -0.10632198, -0.13513198,
         0.15255399, -0.08886399,  0.041814  ,  0.20115899, -0.06696299,
        -0.021407  , -0.018694  ,  0.14587398,  0.010333  , -0.13283598,
        -0.20868897,  0.019673  ,  0.04443499,  0.09935899, -0.12921299,
        -0.001451  , -0.0195    ,  0.012884  ,  0.20407197,  0.048896  ,
         0.027785  ,  0.10837299, -0.031266  ,  0.040904  , -0.03869599,
         0.27307597,  0.15643698,  0.15945998, -0.12120698, -0.26675996,
         0.36759698, -0.14497198,  0.13386598, -0.05709899, -0.15328199,
         0.23343197, -0.15918198, -0.14019199,  0.06286399,  0.13915598]],
      dtype=float32)>

In [759]:
import numpy as np

In [ ]:
tf.keras.layers.Masking()

In [830]:
class PreTrainedEmbedding(tf.keras.layers.Layer):
  
    def __init__(self, mask_value = '', **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.embedding = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50-with-normalization/1",
            dtype=tf.string,
            input_shape=[[]],
           output_shape=[50])
        self.mask_value = mask_value

    def call(self, inputs):
        x = self.embedding(inputs)
        return x

    def compute_mask(self, inputs, mask=None):
        return inputs == self.mask_value

In [835]:
x = [['what', 'is', 'going', 'on']]
x = tf.convert_to_tensor(x, dtype = tf.string)

In [836]:
x

<tf.Tensor: shape=(1, 4), dtype=string, numpy=array([[b'what', b'is', b'going', b'on']], dtype=object)>

In [837]:
model = tf.keras.Sequential([MyLayer()])

In [838]:
model(x)

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([[b'what' b'is' b'going' b'on']], shape=(1, 4), dtype=string))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.string, name=None))

In [782]:
output._keras_mask

<tf.Tensor: shape=(6,), dtype=bool, numpy=array([ True, False, False, False, False, False])>

In [774]:
x == ''

<tf.Tensor: shape=(5,), dtype=bool, numpy=array([False, False, False, False, False])>

In [770]:
layer(x)

In [751]:
embeddings = embed(['',"hi", "is on the mat", "dog is in the fog"])

In [754]:
embeddings[0]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([ 0.3719569 ,  0.12941009, -0.24497539, -0.159289  ,  0.04645426,
       -0.01284458,  0.01933256, -0.11805988, -0.1939171 , -0.01796856,
        0.00275443,  0.04685177, -0.11076233,  0.14695394,  0.10868872,
       -0.07049236,  0.12981366, -0.11780803, -0.16176392,  0.06146137,
        0.12546906,  0.06843865, -0.1838932 ,  0.11003214, -0.13796109,
       -0.36550882, -0.01894941,  0.11452176, -0.12596805,  0.01404112,
       -0.02207962,  0.17831059, -0.07782353,  0.08998444,  0.01423768,
       -0.18583873,  0.00678408, -0.08178326,  0.04413667,  0.07340884,
        0.05071118,  0.38985828, -0.0923743 ,  0.04498108,  0.1979443 ,
       -0.05394636, -0.11659805,  0.04918613,  0.3098403 ,  0.27047452],
      dtype=float32)>

In [749]:
a = ['',"hi", "is on the mat", "dog is in the fog"]

In [8]:
import os

In [50]:
parent_dir = '../data/processed/social/train'
file_name = 'social.txt'

In [193]:
dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))

In [194]:
dataset = dataset.shuffle(1000)

In [182]:
dataset = preprocess_data(dataset, 100, 64)

In [199]:
it = iter(dataset)
next(it)

InvalidArgumentError: slice index 1 of dimension 0 out of bounds.
	 [[{{node strided_slice_1}}]]

In [196]:
dataset = dataset.map(split_labels)

# dataset is fairly small (c. 200mb) so use the cache
# dataset = dataset.cache()

# Shuffle and batch the dataset
# dataset = dataset.shuffle(buffer_size)

In [198]:
dataset = dataset.batch(64)

# # Use prefetch to allow model to get batches in the background while training
# dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [120]:
# Seperate input and outputs
dataset = dataset.map(split_labels)

# dataset is fairly small (c. 200mb) so use the cache
dataset.cache()

# Shuffle and batch the dataset
dataset = dataset.shuffle(1000)
dataset = dataset.batch(64)

# # Use prefetch to allow model to get batches in the background while training
# dataset.prefetch(tf.data.experimental.AUTOTUNE)

# return dataset

In [113]:
def split_labels(data):
    string = tf.strings.split(data, sep='--$--')
    input_text = string[0]
    output_text = string[1]
    return input_text, output_text

In [53]:
data = data.map(split_labels)

In [55]:
it = iter(data)
next(it)

(<tf.Tensor: shape=(), dtype=string, numpy=b"Probably Vinnie's then elsewhere">,
 <tf.Tensor: shape=(), dtype=string, numpy=b'<sos> Got a netball crewers <eos>'>)

In [134]:
import src.data_transform.preprocessing as pp

In [279]:
parent_dir = '../data/processed/social'
file_name = 'social.txt'

In [402]:
def view(dataset):
    for ex in dataset.take(5):
        print(ex)

In [1142]:
dataset = pp.load_from_txt(os.path.join(parent_dir, file_name))

In [1143]:
view(dataset)

tf.Tensor(b'either of you going to supper in hall--$--<sos> Yeah <eos>', shape=(), dtype=string)
tf.Tensor(b"Yeah--$--<sos> You're so paranoid <eos>", shape=(), dtype=string)
tf.Tensor(b"You're so paranoid--$--<sos> mate i had a great lunch alone <eos>", shape=(), dtype=string)
tf.Tensor(b'mate i had a great lunch alone--$--<sos> Awful <eos>', shape=(), dtype=string)
tf.Tensor(b'Awful--$--<sos> I am also <eos>', shape=(), dtype=string)


In [1144]:
with open(os.path.join(parent_dir, file_name)) as f:
    a = f.readlines()

In [1145]:
a = [i.strip() for i in a ]

In [1146]:
dataset, integer_encoder = pp.preprocess_data(dataset.take(100), 1000, 64, vocab_size=20000)

In [1150]:
for ex in dataset.take(1):
    print(integer_encoder.decode(ex[0][0][0]))

fine probably is signature digital a pasting and copy it sign just you can form the out filled have i great ok


In [1070]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50-with-normalization/1")
vocab_list = integer_encoder.tokens
# Add the padding to 0
vocab_list.insert(0, '')
pretrained_embeddings = pp.build_embeddings_from_vocab(vocab_list[:100], embed)


(101, 50)


In [1071]:
pretrained_embeddings.shape

(101, 50)

In [1075]:
pretrained_embeddings[100]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [1024]:
embed(['hello', 'what'])

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[-0.16658899, -0.026856  , -0.23500197,  0.16764398, -0.027478  ,
        -0.05976099,  0.21726097, -0.14739199, -0.10632198, -0.13513198,
         0.15255399, -0.08886399,  0.041814  ,  0.20115899, -0.06696299,
        -0.021407  , -0.018694  ,  0.14587398,  0.010333  , -0.13283598,
        -0.20868897,  0.019673  ,  0.04443499,  0.09935899, -0.12921299,
        -0.001451  , -0.0195    ,  0.012884  ,  0.20407197,  0.048896  ,
         0.027785  ,  0.10837299, -0.031266  ,  0.040904  , -0.03869599,
         0.27307597,  0.15643698,  0.15945998, -0.12120698, -0.26675996,
         0.36759698, -0.14497198,  0.13386598, -0.05709899, -0.15328199,
         0.23343197, -0.15918198, -0.14019199,  0.06286399,  0.13915598],
       [ 0.11003302,  0.17160103, -0.025853  ,  0.23984104, -0.20048703,
        -0.15196803,  0.00338   ,  0.006136  , -0.19012102, -0.0142    ,
        -0.014273  , -0.14131203,  0.02888901, -0.06125001, -0.11314602,
 

In [1036]:
tf.nn.embedding_lookup(
    pretrained_embeddings, 49, max_norm=None, name=None
)

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([ 0.11003302,  0.17160103, -0.025853  ,  0.23984104, -0.20048703,
       -0.15196803,  0.00338   ,  0.006136  , -0.19012102, -0.0142    ,
       -0.014273  , -0.14131203,  0.02888901, -0.06125001, -0.11314602,
        0.09258202, -0.13020404, -0.011828  , -0.19995204,  0.06779601,
       -0.08705802,  0.09041201,  0.09869102,  0.0008    , -0.30639905,
        0.16570203, -0.19105403,  0.012072  ,  0.35205305, -0.20784102,
       -0.14005104, -0.16913003,  0.19045703,  0.08872502,  0.10603502,
        0.10625902,  0.19870003,  0.021559  , -0.17443202, -0.002066  ,
        0.06007201, -0.02526   , -0.05816201,  0.17188203, -0.03096901,
        0.18614003,  0.04651801, -0.21517904,  0.17803103, -0.062681  ],
      dtype=float32)>

In [1042]:
tf.size(pretrained_embeddings)

<tf.Tensor: shape=(), dtype=int32, numpy=5000>

In [1032]:
vocab_list[1]

'<sos>'

In [1097]:
layer = PreTrainedEmbedding(pretrained_embeddings, mask_value=0)

In [1100]:
layer(np.asarray([0, 100]))._keras_mask

<tf.Tensor: shape=(2,), dtype=bool, numpy=array([ True, False])>

In [1045]:
inputs = np.asarray([99, 100, 101])

In [1046]:
oov = inputs >= pretrained_embeddings.shape[0]

In [1048]:
inputs[oov]

array([100, 101])